In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'
manuscript_min_tf = 10
manuscript_max_tf = 0.9
state_path = 's3://ci-elife-data-pipeline/airflow-config/data-science/state-dev'

In [2]:
import os
from functools import partial
from itertools import groupby
from typing import List, Tuple, TypeVar

import joblib
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import data_science_pipeline.configure_warnings

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import to_gbq
from data_science_pipeline.utils.io import load_object_from
from data_science_pipeline.utils.misc import identity_fn
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query
)
from data_science_pipeline.peerscout.models import (
    WeightedKeywordModel
)

In [3]:
model_path = os.path.join(state_path, 'senior_editor_model.joblib')
recommendation_output_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_recommendation'
)

In [4]:
print('loading model from:', model_path)
model_dict = load_object_from(model_path)
model_dict.keys()

loading model from: s3://ci-elife-data-pipeline/airflow-config/data-science/state-dev/senior_editor_model.joblib


dict_keys(['editor_tf_idf_vectorizer', 'editor_tf_idf', 'editor_names', 'editor_person_ids'])

In [5]:
editor_tf_idf_vectorizer = model_dict['editor_tf_idf_vectorizer']
editor_tf_idf_vectorizer

TfidfVectorizer(lowercase=False, smooth_idf=False, token_pattern=None,
                tokenizer=<function identity_fn at 0x7f57f0435d40>)

In [6]:
editor_tf_idf = model_dict['editor_tf_idf']
editor_tf_idf

<65x13658 sparse matrix of type '<class 'numpy.float64'>'
	with 218399 stored elements in Compressed Sparse Row format>

In [7]:
editor_names = model_dict['editor_names']
editor_names

0           Aleksandra Walczak
1                  Andrew King
2               Anna Akhmanova
3     Barbara Shinn-Cunningham
4                Carla Rothlin
                ...           
60                 Tamar Makin
61             Timothy Behrens
62              Utpal Banerjee
63              Vivek Malhotra
64               Wendy Garrett
Name: name, Length: 65, dtype: object

In [8]:
editor_person_ids = model_dict['editor_person_ids']
editor_person_ids

0     50904
1     14601
2      8518
3     19576
4     44396
      ...  
60    18331
61     1044
62     1042
63     1133
64    28627
Name: person_id, Length: 65, dtype: object

In [9]:
editor_person_id_by_name_map = dict(zip(editor_names, editor_person_ids))
editor_person_id_by_name_map

{'Aleksandra Walczak': '50904',
 'Andrew King': '14601',
 'Anna Akhmanova': '8518',
 'Barbara Shinn-Cunningham': '19576',
 'Carla Rothlin': '44396',
 'Catherine Dulac': '1014',
 'Chris Baker': '28129',
 'Christian Büchel': '16197',
 'Christian Hardtke': '1102',
 'Christian Rutz': '15332',
 'Clifford Rosen': '48282',
 'Cynthia Wolberger': '5005',
 'David Ron': '1174',
 'Detlef Weigel': '1030',
 'Didier Stainier': '7189',
 'Diethard Tautz': '1191',
 'Dominique Soldati-Favre': '41647',
 'Eduardo Franco': '1086',
 'Edward Morrisey': '55645',
 'Floris de Lange': '28130',
 'Gary Westbrook': '1202',
 'George Perry': '42011',
 'Gisela Storz': '1188',
 'Huda Zoghbi': '1029',
 'James Manley': '1020',
 'Jessica Tyler': '1421',
 'John Huguenard': '13947',
 'John Kuriyan': '1018',
 'Jonathan Cooper': '1062',
 'Jos van der Meer': '41027',
 'Joshua Gold': '17965',
 'José Faraldo-Gómez': '13987',
 'K VijayRaghavan': '1027',
 'Karla Kirkegaard': '3645',
 'Kate Wassum': '33127',
 'Kathryn Cheah': '91149

In [10]:
weighted_keyword_valid_model = WeightedKeywordModel.from_tf_matrix(
    editor_tf_idf.todense(),
    vectorizer=editor_tf_idf_vectorizer,
    choices=editor_names
)
weighted_keyword_valid_model

In [11]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [12]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [13]:
manuscript_version_for_recommendation_df = read_big_query(
    get_sql('manuscript-version-initial-submissions-for-senior-editor-recommendation.sql').format(
        **default_query_props
    )
)
manuscript_version_for_recommendation_df.head()

> ```sql
> WITH t_manuscript_version_abstract_keywords AS (
>   SELECT
>     manuscript_abstract_keywords.manuscript_id AS manuscript_id,
>     manuscript_abstract_keywords.version_id,
>     manuscript_abstract_keywords.extracted_keywords,
>     ROW_NUMBER() OVER (
>       PARTITION BY version_id
>       ORDER BY data_hub_imported_timestamp DESC
>     ) AS version_id_row_number
>   FROM `elife-data-pipeline.de_dev.manuscript_abstract_keywords` AS manuscript_abstract_keywords
>   WHERE ARRAY_LENGTH(extracted_keywords) > 0
> ),
> 
> t_last_manuscript_version_abstract_keywords AS (
>   SELECT
>     * EXCEPT(version_id_row_number)
>   FROM t_manuscript_version_abstract_keywords
>   WHERE version_id_row_number = 1
>   ORDER BY version_id
> )
> 
> SELECT version.version_id, manuscript_version_abstract_keywords.extracted_keywords
> FROM `elife-data-pipeline.de_dev.mv_manuscript_version` AS version
> JOIN t_last_manuscript_version_abstract_keywords AS manuscript_version_abstract_keywords
>   ON manuscript_version_abstract_keywords.version_id = version.version_id
> WHERE version.overall_stage = 'Initial Submission'
>   AND ARRAY_LENGTH(version.senior_editors) = 0
>   AND TIMESTAMP_DIFF(CURRENT_TIMESTAMP, version.created_timestamp, DAY) < 365
> ```

Downloading: 100%|██████████| 84/84 [00:00<00:00, 91.13rows/s]


,version_id,extracted_keywords
0,49012/2019-06-03T17:44:43Z,"[5,000 toxoplasma, > 5,000 toxoplasma, analysi..."
1,49069/2019-06-05T12:39:51Z,"[1 expression level, accurate, accurate model,..."
2,49311/2019-06-13T15:00:47Z,"[affinity, agonist, amplitude, assembly, biase..."
3,49378/2019-06-15T23:34:46Z,"[- wgs, 16s, 16s rrna gene sequencing, 16s seq..."
4,49391/2019-06-17T03:54:44Z,"[4.1 protein, associate, associated protein, b..."


In [14]:
keyword_similarity = cosine_similarity(
    editor_tf_idf_vectorizer.transform(
        manuscript_version_for_recommendation_df
        ['extracted_keywords']
    ),
    editor_tf_idf
)
print(keyword_similarity.max())
keyword_similarity

0.3135499769748734


array([[0.14662782, 0.08456537, 0.16588939, ..., 0.14509583, 0.13354965,
        0.17615504],
       [0.1747452 , 0.1033788 , 0.1415509 , ..., 0.12053201, 0.12364256,
        0.13381789],
       [0.10326605, 0.09112157, 0.12902313, ..., 0.12395987, 0.13453937,
        0.11132919],
       ...,
       [0.05204138, 0.03480236, 0.03310078, ..., 0.01957362, 0.02554974,
        0.02207618],
       [0.03845917, 0.08417955, 0.0295803 , ..., 0.038947  , 0.02897633,
        0.040758  ],
       [0.04915208, 0.04537317, 0.05262557, ..., 0.0337464 , 0.04276412,
        0.04103865]])

In [15]:
weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords'][:1],
).proba_matrix

matrix([[0.14662782, 0.08456537, 0.16588939, 0.06535409, 0.0647289 ,
         0.12971141, 0.04044395, 0.07769449, 0.14635631, 0.08110631,
         0.11198752, 0.12275343, 0.11348325, 0.18347194, 0.17149085,
         0.14015457, 0.31354998, 0.10763166, 0.12172587, 0.07163061,
         0.0962416 , 0.02958908, 0.1793357 , 0.11900766, 0.15510872,
         0.15330543, 0.06500295, 0.12178744, 0.14272576, 0.10862883,
         0.0702276 , 0.09920733, 0.14834451, 0.14711333, 0.06581512,
         0.14914818, 0.07361117, 0.16913675, 0.08398592, 0.17675513,
         0.07515114, 0.13687253, 0.14180001, 0.07470227, 0.12890685,
         0.07103443, 0.20684821, 0.13514644, 0.09290031, 0.16885972,
         0.14115072, 0.08920739, 0.14356624, 0.09767392, 0.07391653,
         0.12792811, 0.09076624, 0.15208646, 0.11134478, 0.15266891,
         0.04863221, 0.07870388, 0.14509583, 0.13354965, 0.17615504]])

In [16]:
manuscript_matching_keywords_list = weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords']
).matching_keywords_list
pd.Series(manuscript_matching_keywords_list[:5])

0    [[(0.018472135321883926, cell), (0.00855469548...
1    [[(0.02007083167833441, cell), (0.017745140061...
2    [[(0.01664526780690731, cell), (0.007708648447...
3    [[(0.0068517406132837175, result), (0.00546514...
4    [[(0.02599055962139287, cell), (0.012036579368...
dtype: object

In [17]:
T = TypeVar('T')


def get_recommended_editors_with_probability(
        proba_matrix: List[List[float]],
        manuscript_matching_keywords_list: List[List[Tuple[float, str]]],
        indices: List[T],
        threshold: float = 0.5) -> List[List[Tuple[float, T]]]:
    return [
        sorted([
            (
                p,
                key,
                sum(
                    s for s, _ in editor_matching_keywords
                ),
                editor_matching_keywords
            )
            for p, key, editor_matching_keywords in zip(
                row,
                indices,
                editors_matching_keywords
            ) if p >= threshold
        ], reverse=True)
        for row, editors_matching_keywords in zip(proba_matrix, manuscript_matching_keywords_list)
    ]


prediction_results_with_similarity = pd.Series(
    get_recommended_editors_with_probability(
        keyword_similarity,
        manuscript_matching_keywords_list,
        editor_names,
        threshold=0.001
    ),
    index=manuscript_version_for_recommendation_df.index
)
# print(prediction_results_with_similarity[0])
prediction_results_with_similarity[:5]

0    [(0.3135499769748734, Dominique Soldati-Favre,...
1    [(0.20783742798234378, Naama Barkai, 0.2078374...
2    [(0.17022937944810723, Tadatsugu Taniguchi, 0....
3    [(0.27923263049409064, Wendy Garrett, 0.279232...
4    [(0.28822596241125276, Dominique Soldati-Favre...
dtype: object

In [18]:
prediction_results_df = pd.concat([
    manuscript_version_for_recommendation_df['version_id'],
    prediction_results_with_similarity.to_frame('prediction'),
], axis=1)
print(len(prediction_results_df))
prediction_results_df.head()

84


,version_id,prediction
0,49012/2019-06-03T17:44:43Z,"[(0.3135499769748734, Dominique Soldati-Favre,..."
1,49069/2019-06-05T12:39:51Z,"[(0.20783742798234378, Naama Barkai, 0.2078374..."
2,49311/2019-06-13T15:00:47Z,"[(0.17022937944810723, Tadatsugu Taniguchi, 0...."
3,49378/2019-06-15T23:34:46Z,"[(0.27923263049409064, Wendy Garrett, 0.279232..."
4,49391/2019-06-17T03:54:44Z,"[(0.28822596241125276, Dominique Soldati-Favre..."


In [19]:
prediction_results_df['prediction'][0]

[(0.3135499769748734,
  'Dominique Soldati-Favre',
  0.3135499769748719,
  [(0.0397204424435791, 'parasite'),
   (0.019399919354767477, 'protozoan'),
   (0.019399919354767477, 'toxoplasma gondii'),
   (0.0171356667864739, 'gondii'),
   (0.015307770176391879, 'toxoplasma'),
   (0.014279722322061582, 'asexual'),
   (0.014246192288124907, 'host'),
   (0.012334317042091179, 'cell'),
   (0.012153515476704785, 'infection'),
   (0.008743418051715395, 'protozoan parasite'),
   (0.007435170877682888, 'scrna-seq'),
   (0.006440016626665503, 'gene'),
   (0.00616082129861707, 'life cycle'),
   (0.006091376737471827, 'surface'),
   (0.005712652254607501, 'stage'),
   (0.005497187609825607, 'antigen'),
   (0.004757863121257025, 'complex'),
   (0.004613691030419352, 'study'),
   (0.004256725305354964, 'cycle'),
   (0.00414001068857068, 'type'),
   (0.004014471666121632, 'development'),
   (0.003892797099210293, 'analysis'),
   (0.003795174474489593, 'population'),
   (0.0037748381157976514, 'result')

In [20]:
prediction_results_flat_df = pd.DataFrame([
    {
        'version_id': row.version_id,
        'score': predicted_editor[0],
        'name': predicted_editor[1],
        'person_id': editor_person_id_by_name_map[predicted_editor[1]],
        'matching_keyword_score': predicted_editor[2],
        'matching_keywords': [{
            'score': keyword_score,
            'keyword': keyword
        } for keyword_score, keyword in predicted_editor[3]],
    }
    for row in prediction_results_df.itertuples()
    for predicted_editor in row.prediction
])
print(len(prediction_results_flat_df))
prediction_results_flat_df.head()

5386


,version_id,score,name,person_id,matching_keyword_score,matching_keywords
0,49012/2019-06-03T17:44:43Z,0.313550,Dominique Soldati-Favre,41647,0.313550,"[{'score': 0.0397204424435791, 'keyword': 'par..."
1,49012/2019-06-03T17:44:43Z,0.206848,Naama Barkai,1725,0.206848,"[{'score': 0.02338325933106585, 'keyword': 'ce..."
2,49012/2019-06-03T17:44:43Z,0.183472,Detlef Weigel,1030,0.183472,"[{'score': 0.01794872332040266, 'keyword': 'ge..."
3,49012/2019-06-03T17:44:43Z,0.179336,Gisela Storz,1188,0.179336,"[{'score': 0.016344050852180488, 'keyword': 'c..."
4,49012/2019-06-03T17:44:43Z,0.176755,Marianne Bronner,1052,0.176755,"[{'score': 0.024457210590623626, 'keyword': 'c..."


In [21]:
prediction_results_flat_df['version_id'].nunique()

84

In [22]:
prediction_results_flat_df.max()

version_id                58803/2020-05-11T11:55:13Z
score                                        0.31355
name                                   Wendy Garrett
person_id                                      97155
matching_keyword_score                       0.31355
dtype: object

In [23]:
print('writing to:', recommendation_output_table_name)
to_gbq(
    prediction_results_flat_df,
    recommendation_output_table_name,
    project_id=project_id,
    if_exists='replace'
)
print('done')

writing to: de_dev.data_science_editor_recommendation
done
